In [2]:
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm
import datetime

def add_hours_to_iso_time(iso_time):
    """Adds 3*steps hours to the given iso_time.

    Arguments:
        iso_time [string]: The ISO time string to be extended.
        steps [int]: The number of steps to extend the iso_time by (each step is 3h).

    Returns:
        [int]: The corresponding id.
    """
    day = int(iso_time.split('/')[0])
    month = int(iso_time.split('/')[1])
    year = int(iso_time.split('/')[2][:4])
    hour = int(iso_time[-5:-3])
    date = datetime.datetime(year, month, day, hour)
    extended_date = date + datetime.timedelta(hours=-1, minutes=-30)

    extended_iso_time = f'{extended_date.day}/{extended_date.month}/{extended_date.year} {extended_date.hour:02d}:{extended_date.minute}'
    return extended_iso_time

ibtracs = pd.read_csv('data/IBTrACS/ibtracs.nangka.csv')
ibtracs.head()
ibtracs_interp = []
ibtracs_interp.append(ibtracs.iloc[0])

previous_lat = ibtracs.LAT[0]
previous_lon = ibtracs.LON[0]

for idx, row in tqdm(ibtracs.iloc[1:].iterrows(), total=ibtracs.shape[0]-1):
    middle_row = row.copy()
    current_lat = row.LAT
    current_lon = row.LON
    current_iso = row.ISO_TIME
    middle_row['LAT'] = (current_lat + previous_lat) / 2
    middle_row['LON'] = (current_lon + previous_lon) / 2
    middle_row['ISO_TIME'] = add_hours_to_iso_time(current_iso)
    ibtracs_interp.append(middle_row)
    previous_lat = current_lat
    previous_lon = current_lon

100%|██████████| 29/29 [00:00<00:00, 11509.73it/s]


In [3]:
new_df = pd.DataFrame(ibtracs_interp, columns = ibtracs.columns.to_list())
new_df.to_csv(f'data/IBTrACS/ibtracs.nangka_interp.csv', index=False)